In [ ]:
import torch
import torch.nn as nn

class CRF_decoder(nn.Module):
    def __init__(self, num_class, h_size=768, t_weight=1):

        self.transitions = \
            nn.Parameter(torch.randn(num_class, num_class))
            
        self.start_transitions = \
            nn.Parameter(torch.randn(1, num_class))
        
        self.classifier = nn.Linear(h_size, num_class)
        
    def forward(self, encoder_outputs, encoder_mask, labels):
        emissision = self.classifier(encoder_outputs) #[B, N, C]
        transition = self.classifier(encoder_outputs) #[B, N, C]
        
        
    def forward_alg():
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha


In [ ]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
class CRF(torch.nn.Module):
    def __init__(self, embeddings, embedding_size, hidden_size, label_number):
        super().__init__()
        self.label_number = label_number
        self.embedding = torch.nn.Embedding.from_pretrained(embeddings)
        self.LSTM = torch.nn.LSTM(embedding_size, hidden_size, 1, False, True, 0, True)
        self.classifier = torch.nn.Linear(2*hidden_size, 50, bias = False)
        self.classifier_2 = torch.nn.Linear(50, label_number, bias = False)
        self.tanh = torch.nn.Tanh()
        self.dropout = torch.nn.Dropout(0.5)

        self.transitions = torch.nn.Parameter(torch.randn(label_number*label_number,1))
        self.lambd = torch.nn.Parameter(torch.randn(1))

        self.device = torch.device("cuda")

        #self.transitions = torch.FloatTensor([[10],[9],[5],[7]]).to(torch.device("cuda"))
        #self.transitions = torch.zeros(label_number*label_number,1).to(self.device)

    def forward(self, sentence_ids, mask, labels):
        context_tensor = self.encode(sentence_ids, mask)

        emission_score_mat = self._get_emission_scores(context_tensor, mask)
        sentence_score_mat = self._get_sentence_scores(emission_score_mat, labels)
        normalising_mat = self._get_partition_function(emission_score_mat)
        sentence_scores = torch.sum(torch.mul(sentence_score_mat, mask), dim=-1)
        sum_sentence_score = torch.sum(sentence_scores, dim=-1)

        sum_Z_score = torch.zeros(1).to(self.device)
        mask_seq = torch.sum(mask.to(torch.long), dim=-1)
        for i in range(len(normalising_mat)):
            sequence = normalising_mat[i]
            sum_Z_score += sequence[mask_seq[i]-1]

        return -(sum_sentence_score - sum_Z_score)

    def decode(self, sentence_ids, mask):
        context_tensor = self.encode(sentence_ids, mask)
        emission_score_mat = self._get_emission_scores(context_tensor, mask)

        decision_matrix = torch.zeros(sentence_ids.shape[0], sentence_ids.shape[1]).to(self.device)
        mask_sequence = torch.sum(mask, dim=1).int().tolist()
        num_label = self.label_number
        for i in range(len(sentence_ids)):
            best_list = [[0, []] for _ in range(num_label)]
            extended_list = [[0, []] for _ in range(num_label*num_label)]

            for j in range(mask_sequence[i]):
                emission = emission_score_mat[i][j]
                for k in range(len(extended_list)):
                    extended_list[k][0] = best_list[k//num_label][0] + self.transitions[k] + emission[k%num_label]
                    extended_list[k][1] = best_list[k//num_label][1] + [k%num_label]

                for m in range(len(best_list)):
                    paths = [b for a,b in enumerate(extended_list) if a%num_label == m]
                    best_list[m] = max([path for path in paths]).copy()

            decoded_sequence = max(best_list)[1]
            decision_matrix[i,:len(decoded_sequence)] = torch.FloatTensor(decoded_sequence)

        sentence_score_mat = self._get_sentence_scores(emission_score_mat, decision_matrix)
        cum_f_score = torch.cumsum(sentence_score_mat, dim=-1)
        normalising_mat = self._get_partition_function(emission_score_mat)

        shifted_f = torch.cat((torch.zeros_like(cum_f_score[:,-1:]), cum_f_score[:,:-1]),1)
        shifted_Z = torch.cat((torch.zeros_like(normalising_mat[:,-1:]), normalising_mat[:,:-1]),1)

        log_p = (cum_f_score - normalising_mat) - (shifted_f - shifted_Z)
        probs = torch.exp(log_p.unsqueeze(-1))
        result = probs * one_hot_encoding(decision_matrix)
        result[:,:,0][result[:,:,0] == 0] = 1
        result[:,:,0]  = 1 - result[:,:,0]
        output = torch.sum(result, dim=-1)
        return one_hot_encoding(output)

    def encode(self, sentence_ids, mask):
        input_tensor = self.embedding(sentence_ids)
        mask_sequence = torch.sum(mask, dim=1)
        jagged_input_tensor = torch.nn.utils.rnn.pack_padded_sequence(input_tensor, mask_sequence, batch_first = True, enforce_sorted = False)
        jagged_context_embedding_tensor, _ = self.LSTM(jagged_input_tensor)
        context_tensor, _  = torch.nn.utils.rnn.pad_packed_sequence(jagged_context_embedding_tensor, batch_first = True, total_length = input_tensor.size(1))
        context_tensor = self.dropout(context_tensor)
        return context_tensor


    def _get_emission_scores(self, context_tensor, mask):
        output_1 = self.classifier(context_tensor)
        output_2 = self.classifier_2(torch.tanh(output_1))
        return output_2

    def _get_sentence_scores(self, GED_probs, labels):
        emission_matrix = torch.sum(torch.mul(one_hot_encoding(labels),GED_probs), -1)
        scores = torch.zeros_like(labels).to(self.device)
        binary_converter = torch.tensor([[self.label_number],[1.0]]).to(self.device)
        batch_size = labels.shape[0]
        scores[:, 0] = emission_matrix[:,0].view(batch_size)
        for i in range(1, GED_probs.shape[1]):
            transition_labels = torch.matmul(labels[:, i-1:i+1], binary_converter).to(torch.long)
            emission_scores = emission_matrix[:, i:i+1]
            scores[:, i] = self.transitions[transition_labels].view(batch_size) + emission_scores.view(batch_size)
        return scores

    def _log_sum_exp(self, tensor):
        max_tensor, _ = torch.max(tensor, -1)
        tensor = tensor - max_tensor.unsqueeze(-1)
        return max_tensor + torch.log(torch.sum(torch.exp(tensor), dim=-1))

    def _TEMP_partition_function(self, GED_probs):
        GED_probs = GED_probs[:1,:, :]
        sum_TEMP = torch.zeros(1).to(self.device)
        sum_TEMP_2 = torch.zeros(1).to(self.device)
        sum_TEMP_3 = torch.zeros(1).to(self.device)

        for i in [0,1]:
            temp_labels = torch.FloatTensor([[i]]).to(self.device)
            sum_TEMP_3 += torch.exp(torch.sum(self._get_sentence_scores(GED_probs[:,:1,:], temp_labels)))
            for j in [0,1]:
                temp_labels = torch.FloatTensor([[i,j]]).to(self.device)
                sum_TEMP_2 += torch.exp(torch.sum(self._get_sentence_scores(GED_probs[:,:2,:], temp_labels)))
                for k in [0,1]:
                    for l in [0,1]:
                        for m in [0,1]:
                            temp_labels = torch.FloatTensor([[i,j,k,l,m]]).to(self.device)
                            print(temp_labels, self._get_sentence_scores(GED_probs, temp_labels))
                            sum_TEMP += torch.exp(torch.sum(self._get_sentence_scores(GED_probs, temp_labels)))
        print(torch.log(sum_TEMP))
        print(torch.log(sum_TEMP_2))
        print(torch.log(sum_TEMP_3))
        pdb.set_trace()

    def _get_partition_function(self, GED_probs):
        transitions = self.transitions.view(1,1,self.label_number,self.label_number)
        GED_probs = GED_probs.unsqueeze(-1)
        forward_matrix = GED_probs + transitions.permute(0,1,3,2)
        Z = torch.zeros_like(GED_probs)
        Z[:,0] = GED_probs[:,0]
        for i in range(GED_probs.shape[1]-1):
            update = self._log_sum_exp(forward_matrix[:,i+1] + Z[:, i].permute(0,2,1))
            Z[:, i+1] = update.unsqueeze(-1)

        Z = Z.squeeze(-1)
        Z = self._log_sum_exp(Z)
        return (Z)